In [1]:
import pandas as pd
import os
import json
import requests

dataset_name = "Drone"
dataset_description = "UAV controller"

PATH_TO_DATASET = "./%s" % dataset_name

def read_dataset_file(file_name):
    path_to_file = os.path.join(PATH_TO_DATASET, file_name)
    with open(path_to_file) as f:
        data = json.load(f)
    return data

In [2]:
requirements = read_dataset_file("requirements.json")["artifacts"]
designs = read_dataset_file("designs.json")["artifacts"]
classes = read_dataset_file("classes.json")["artifacts"]
artifacts = requirements + designs + classes
artifacts = list(filter(lambda a: "name" in a, artifacts))
artifact_map = {}
for a in artifacts:
    a.update({"type":a['type'].lower()})
    artifact_map[a['name']] = 1
artifacts[:3]

[{'name': 'RE-8',
  'body': 'UAV State transitions When requested the _VehicleCore_ shall shall transition the UAV between states according to allowed state transitions as depicted in the UAV state transition diagram',
  'type': 'requirements'},
 {'name': 'RE-9',
  'body': 'ListenerNotifications If a client registers for notifications the _UIMiddleware_ shall send notifications for specific events',
  'type': 'requirements'},
 {'name': 'RE-25',
  'body': 'UAV Identification The _VehicleCore_ shall assign a unique ID to each activated UAV.',
  'type': 'requirements'}]

In [3]:
def get_artifact_types(matrix_name):
    name_parts = matrix_name.split("to")
    return get_name_type(name_parts[0]), get_name_type(name_parts[1])
    
def get_name_type(name):
    if "1" in name:
        return "requirements"
    elif "2" in name:
        return "designs"
    elif "3" in name:
        return "code"
    else:
        raise Exception("unknown type: %s" % name)

In [4]:
assert get_artifact_types('Level_2_to_Level_3.csv') == ("designs", "code")

In [6]:
PATH_TO_TRACE_MATRICES = os.path.join(PATH_TO_DATASET, "TraceMatrices")
trace_matrix_names = list(filter(lambda f: f[0] != ".", os.listdir(PATH_TO_TRACE_MATRICES)))
traces = []
for trace_matrix_name in trace_matrix_names:
    path_to_matrix = os.path.join(PATH_TO_TRACE_MATRICES, trace_matrix_name)
    trace_matrix = pd.read_csv(path_to_matrix)
    
    source_type, target_type = get_artifact_types(trace_matrix_name)
    
    matrix = trace_matrix
    
    for i in range(len(matrix)):
        for col in matrix.columns:
            if matrix.iloc[i][col] == 1:
                source_name = matrix["id"][i]
                target_name = col
                
                if source_name in artifact_map and target_name in artifact_map:
                    trace = {
                        "source_type": source_type,
                        "target_type": target_type,
                        "source_name": source_name,
                        "target_name": target_name
                    }
                    traces.append(trace)


In [7]:
dataset_data = {
    "project":{
        "name": dataset_name,
        "description": dataset_description,
    },
    "artifacts": artifacts,
    "traces": traces
}
response = requests.post("http://localhost:8000/projects", json = dataset_data)
response.text

'{"id":"443a4015-f204-4fc8-82fb-f9206f61c756","name":"Drone","artifacts":[{"id":"f4a48ded-6cc8-4213-a214-a2671c32d8e9","type":"designs","name":"DD-11","body":"Client registers for flight route events A client shall register with the _UIMiddleware_ to receive flight route event notifications whenever a flight route is created modified or deleted."},{"id":"b3ff567a-bf56-438e-8f45-784b18a60b36","type":"designs","name":"DD-113","body":"Display state of activated UAVs The name state battery level speed and current coordinates of each UAV shall be displayed."},{"id":"164fd1e4-bd35-4136-9194-2dceffaaa2ac","type":"designs","name":"DD-118","body":"Load saved map scale When loading the map it shall be scaled as specified in the configuration."},{"id":"3a91b82b-63aa-4f81-b9b6-ce4960b206ed","type":"designs","name":"DD-121","body":"Update UAV location frequency The UAV location shall be updated on the map at the currently defined {{UI_UAV_REFRESH_RATE}}"},{"id":"ccdfa240-49fa-4952-b4a6-74832d6fed2a

In [8]:
traces

[{'source_type': 'requirements',
  'target_type': 'designs',
  'source_name': 'RE-576',
  'target_name': 'DD-577'},
 {'source_type': 'requirements',
  'target_type': 'designs',
  'source_name': 'RE-80',
  'target_name': 'DD-529'},
 {'source_type': 'requirements',
  'target_type': 'designs',
  'source_name': 'RE-80',
  'target_name': 'DD-527'},
 {'source_type': 'requirements',
  'target_type': 'designs',
  'source_name': 'RE-595',
  'target_name': 'DD-606'},
 {'source_type': 'requirements',
  'target_type': 'designs',
  'source_name': 'RE-595',
  'target_name': 'DD-601'},
 {'source_type': 'requirements',
  'target_type': 'designs',
  'source_name': 'RE-77',
  'target_name': 'DD-23'},
 {'source_type': 'requirements',
  'target_type': 'designs',
  'source_name': 'RE-508',
  'target_name': 'DD-509'},
 {'source_type': 'requirements',
  'target_type': 'designs',
  'source_name': 'RE-25',
  'target_name': 'DD-573'},
 {'source_type': 'requirements',
  'target_type': 'designs',
  'source_name':